In [5]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



###Dataset

In [6]:
iris = load_iris()
X = iris.data
y = iris.target


In [7]:
X.shape

(150, 4)

In [9]:
y.shape

(150,)

In [10]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

X = X.astype(np.float32)
y = y.astype(np.int64)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [19]:
class IrisDataset(Dataset):
  def __init__(self,X,y):
    self.X = torch.tensor(X)
    self.y = torch.tensor(y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

In [20]:
train_dataset = IrisDataset(X_train,y_train)
test_dataset = IrisDataset(X_test,y_test)

train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=16)

###MLP Model

In [24]:
class MLP(nn.Module):
  def __init__(self,input_dim,num_classes):
    super(MLP,self).__init__()
    self.model = nn.Sequential(
        nn.Linear(input_dim,32),
        nn.ReLU(),
        nn.Linear(32,16),
        nn.ReLU(),
        nn.Linear(16,num_classes)
    )

  def forward(self,x):
    return self.model(x)

model = MLP(input_dim=4,num_classes=3)

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

###Training Loop

In [26]:
epochs = 100
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  for inputs,targets in train_loader:
    outputs = model(inputs)
    loss = criterion(outputs,targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  if (epoch+1) % 10 == 0:
    print(f"Epoch: {epoch+1}/{epochs}, Loss: {running_loss: .4f}")

Epoch: 10/100, Loss:  0.5513
Epoch: 20/100, Loss:  0.3788
Epoch: 30/100, Loss:  0.4924
Epoch: 40/100, Loss:  0.3182
Epoch: 50/100, Loss:  0.3110
Epoch: 60/100, Loss:  0.2268
Epoch: 70/100, Loss:  0.2753
Epoch: 80/100, Loss:  0.1514
Epoch: 90/100, Loss:  0.1204
Epoch: 100/100, Loss:  0.0480


###Evaluation

In [28]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for inputs,targets in test_loader:
    outputs = model(inputs)
    _, predicted  = torch.max(outputs,1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()

accuracy = correct / total * 100
print(f"Accuracy : {accuracy}")

Accuracy : 100.0
